In [1]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange
from copy import deepcopy

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

In [60]:
#设置value的显示长度为200，默认为50
pd.set_option('max_colwidth',200)
#显示所有列，把行显示设置成最大
pd.set_option('display.max_columns', None)
#显示所有行，把列显示设置成最大
pd.set_option('display.max_rows', None)

In [2]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
def read_data(type_ = 'train', fold = None):
    if type_ != 'train' and type_ != 'val':
        data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/train_data_fold{}.csv'.format(fold), index_col = 0)
    elif type_ == 'val':
        data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/val_data_fold{}.csv'.format(fold), index_col = 0)
    return data

In [5]:
new_type = 'independent'
ori_type = 'evaluation'
data = read_data(new_type, None)
print('{} set Shape: '.format(new_type), data.shape)
data.head()

independent set Shape:  (171438, 5)


,peptide,length,HLA,label,HLA_sequence
0,LFGRDLSY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
1,TDKKTHLY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
2,RSDTPLIY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
3,NSDLVQKY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
4,LSDLLDWK,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY


In [6]:
methods = ['ann', 
           'comblib_sidney2008', 
           'consensus', 
           'netmhccons', 
           'netmhcpan_ba', 
           'netmhcpan_el', 
           'netmhcstabpan', 
           'pickpocket', 
           'smm', 
           'smmpmbec',
           'anthem',
           'transformer']

In [7]:
hla_methods_dict = {}
for method in methods[:-2]:
    hlas_filename = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/api_provide_hlas/' + method + '.hlas'
    with open(hlas_filename, 'r') as f:
        hlas = f.readlines()
        hlas = [item.replace('\n', '') for item in hlas[1:-3]]
    hla_methods_dict[method] = hlas
    print('Method = {} | # HLAs = {}'.format(method, len(hlas)))

Method = ann | # HLAs = 89
Method = comblib_sidney2008 | # HLAs = 14
Method = consensus | # HLAs = 81
Method = netmhccons | # HLAs = 2915
Method = netmhcpan_ba | # HLAs = 10386
Method = netmhcpan_el | # HLAs = 10386
Method = netmhcstabpan | # HLAs = 2915
Method = pickpocket | # HLAs = 2924
Method = smm | # HLAs = 81
Method = smmpmbec | # HLAs = 83


# Transformer的结果

In [8]:
pHLAIformer_metrics_length_hla_pd_dict = np.load('./results/pHLAIformer_metrics_length_hla_pd_dict.npy', 
                                                 allow_pickle = True).item()[new_type]
pHLAIformer_metrics_pd = pHLAIformer_metrics_length_hla_pd_dict['all']
pHLAIformer_metrics_pd

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
0,0.97888,0.930844,0.861688,0.930927,0.930341,0.931348,0.931513,0.930341,0.978183


In [9]:
pHLAIformer_data = np.load('./results/pHLAIformer_data_res_pd_dict.npy', allow_pickle = True).item()[new_type]
pHLAIformer_data

,peptide,length,HLA,label,HLA_sequence,y_true,y_pred,y_prob
0,LFGRDLSY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,9.999058e-01
1,TDKKTHLY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,9.999999e-01
2,RSDTPLIY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000e+00
3,NSDLVQKY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000e+00
4,LSDLLDWK,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,9.996661e-01
...,...,...,...,...,...,...,...,...
171433,RVLLYNHNLL,10,HLA-B*44:27,0,YYTKYREISTNTYENTAYIRYDDYTWAVDAYLSY,0,0,1.109294e-11
171434,LGTIIFAHCQ,10,HLA-B*44:27,0,YYTKYREISTNTYENTAYIRYDDYTWAVDAYLSY,0,0,1.026913e-09
171435,PPASPTIIPR,10,HLA-B*44:27,0,YYTKYREISTNTYENTAYIRYDDYTWAVDAYLSY,0,0,1.143919e-05
171436,QPNHTPAGPP,10,HLA-B*44:27,0,YYTKYREISTNTYENTAYIRYDDYTWAVDAYLSY,0,0,4.892460e-09


# Anthem的结果

In [10]:
anthem_results_pd = pd.read_csv('/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/Anthem_' + ori_type + '_res_pd.csv', index_col = 0)
anthem_results_pd = pd.merge(pHLAIformer_data, anthem_results_pd, how = 'inner', on = ['peptide', 'HLA', 'length']).reset_index(drop = True)
print('Anthem shape: ', anthem_results_pd.shape)
anthem_results_pd.head()

Anthem shape:  (169523, 10)


,peptide,length,HLA,label,HLA_sequence,y_true,y_pred,y_prob,y_score_pred,score
0,LFGRDLSY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999906,0,0.819
1,TDKKTHLY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000,1,0.999
2,RSDTPLIY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000,1,0.999
3,NSDLVQKY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000,1,0.999
4,LSDLLDWK,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999666,1,0.999


In [13]:
# anthem_metrics_length_pd_dict = np.load('/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/Anthem_' + ori_type + '_metrics_length_pd_dict.npy', allow_pickle = True).item()
# anthem_metrics = anthem_metrics_length_pd_dict['all']
# anthem_metrics

# 结果处理

In [11]:
def performances(y_true, y_pred, y_prob = None, rank_or_ic50 = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = 0
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = 0
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = 0
    
    if y_prob is None:
        roc_auc, aupr = np.nan, np.nan
    else:
        if rank_or_ic50:
            roc_auc = roc_auc_score(1 - y_true, y_prob)
            prec, reca, _ = precision_recall_curve(1 - y_true, y_prob)
        else:
            roc_auc = roc_auc_score(y_true, y_prob)
            prec, reca, _ = precision_recall_curve(y_true, y_prob)
            
        aupr = auc(reca, prec)

    print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
    print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
    print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))

    return roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr

In [12]:
def benchmark_results_pd(pwd, filenames):
    error_files = []
    for idx, filename in enumerate(filenames):
        if idx == 0:
            benchmark_pd = pd.read_csv(pwd + filename, sep = '\t')
            num_col = benchmark_pd.shape[1]
        else:
            try:
                temp = pd.read_csv(pwd + filename, sep = '\t')
            except:
                error_files.append(filename)
                continue

            if temp.shape[1] != num_col: 
                error_files.append(filename)
                continue
            else:
                benchmark_pd = pd.concat([benchmark_pd, temp], axis = 0)
    try:
        assert (benchmark_pd.start == 1).all()
    except:
        print(error_files)
    benchmark_pd.reset_index(drop = True, inplace = True)
    benchmark_pd.drop(columns = ['seq_num', 'start', 'end'], inplace = True)
    benchmark_pd.rename(columns = {'allele':'HLA'}, inplace = True)
    return benchmark_pd, error_files

In [13]:
def merge_benchmark_with_original_data(eval_data, hla_methods_dict, ori_type = 'experimental', method = 'ann'):
    pwd = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/output_file/' + method + '/'
    filenames = sorted(os.listdir(pwd))
    filenames = [name for name in filenames if ori_type in name]
    filenames = [item for item in filenames if item.split('_')[2] in hla_methods_dict[method]]
    if method == 'smm' or method == 'smmpmbec':
        filenames = [item for item in filenames if ('length12' not in item) and ('length13' not in item) and ('length14' not in item)]
    if method == 'comblib_sidney2008':
        filenames = [item for item in filenames if 'length9' in item]

    benchmark_pd, error_files = benchmark_results_pd(pwd, filenames)
    merged_data = pd.merge(eval_data, benchmark_pd, on = ['peptide', 'HLA', 'length'])
    
    if 'consensus_percentile_rank' in merged_data.columns: 
        merged_data.rename(columns = {'consensus_percentile_rank':'percentile_rank'}, inplace = True)
    return merged_data, error_files

In [35]:
def benchmark_label_results(merged_data, method = 'ann'):
    columns = merged_data.columns
    if 'ic50' in columns: 
        print('-----Performances according to IC50 threshold-----')
        merged_data['y_pred_ic50'] = [[0, 1][ic50 <= 500] for ic50 in merged_data.ic50]
        performance = performances(merged_data.label, merged_data.y_pred_ic50, merged_data.ic50, rank_or_ic50 = True)

    elif 'score' in columns: 
        print('-----Performances according to Score threshold-----')
        merged_data['y_pred_score'] = [[0, 1][score >= 0.5] for score in merged_data.score]
        performance = performances(merged_data.label, merged_data.y_pred_score, merged_data.score, rank_or_ic50 = False)
    
    elif 'percentile_rank' in columns: 
        print('-----Performances according to Rank threshold-----')
        merged_data['y_pred_rank'] = [[0, 1][r <= 2] for r in merged_data.percentile_rank]
        performance = performances(merged_data.label, merged_data.y_pred_rank, merged_data.percentile_rank, rank_or_ic50 = True)
    
    return merged_data, performance

In [54]:
def eval_metrics_length_hla_dict(data_eval, lengths, metrics_type):

    rank_or_ic50 = [True, False][metrics_type[0] == 'score']
    
    pHLAIformer_metrics_length_hla_dict = {}
    metrics_length_hla_dict = {}
    for length in lengths:
        data_length = data_eval[data_eval.length == length]
        hlas_length = list(set(data_length.HLA))
        pHLAIformer_metrics_length_hla_dict[length], metrics_length_hla_dict[length] = {}, {}
        for hla in tqdm(hlas_length):
            data_length_hla = data_length[data_length.HLA == hla]
            data_num = data_length_hla.shape[0]
            metrics_length_hla = performances(data_length_hla.label, 
                                              data_length_hla[metrics_type[1]], 
                                              data_length_hla[metrics_type[0]], 
                                              rank_or_ic50)
            pHLAIformer_length_hla = performances(data_length_hla.y_true, 
                                                  data_length_hla.y_pred, 
                                                  data_length_hla.y_prob, 
                                                  False)
            metrics_length_hla_dict[length][hla] = list(metrics_length_hla) + [data_num]
            pHLAIformer_metrics_length_hla_dict[length][hla] = list(pHLAIformer_length_hla) + [data_num]
    return metrics_length_hla_dict, pHLAIformer_metrics_length_hla_dict

def eval_performances_to_pd(metrics_length_dict, length):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr', 'num']
    
    performances_pd = pd.DataFrame(metrics_length_dict[length], index = metrics_name).T
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['median'] = performances_pd.median(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    performances_pd.loc['sum', 'num'] = performances_pd['num'].sum(axis = 0)
    performances_pd['num'] = performances_pd['num'].astype('int')
    
    return performances_pd

# 性能评估

In [21]:
merged_data_dict, error_files_dict = {}, {}
for method in tqdm(methods[:-2]):
    merged_data_dict[method], error_files_dict[method] = merge_benchmark_with_original_data(pHLAIformer_data, hla_methods_dict, ori_type, method)

for k, v in error_files_dict.items():
    print(k, v)

100%|██████████| 10/10 [00:27<00:00,  2.76s/it]

['evaluation_length10_HLA-A*69:01_0.output', 'evaluation_length10_HLA-B*14:02_0.output', 'evaluation_length10_HLA-B*27:03_0.output', 'evaluation_length10_HLA-B*39:01_0.output', 'evaluation_length10_HLA-B*46:01_0.output', 'evaluation_length10_HLA-C*03:03_0.output', 'evaluation_length10_HLA-C*04:01_0.output', 'evaluation_length10_HLA-C*05:01_0.output', 'evaluation_length10_HLA-C*06:02_0.output', 'evaluation_length10_HLA-C*07:01_0.output', 'evaluation_length10_HLA-C*07:02_0.output', 'evaluation_length10_HLA-C*08:02_0.output', 'evaluation_length10_HLA-C*14:02_0.output', 'evaluation_length11_HLA-A*32:01_0.output', 'evaluation_length11_HLA-A*68:01_0.output', 'evaluation_length11_HLA-B*15:01_0.output', 'evaluation_length11_HLA-B*27:03_0.output', 'evaluation_length11_HLA-B*35:03_0.output', 'evaluation_length11_HLA-B*39:01_0.output', 'evaluation_length11_HLA-B*40:01_0.output', 'evaluation_length11_HLA-B*44:02_0.output', 'evaluation_length11_HLA-B*46:01_0.output', 'evaluation_length11_HLA-B*54:0

In [23]:
merged_data_dict['anthem'] = anthem_results_pd
merged_data_dict['anthem']

,peptide,length,HLA,label,HLA_sequence,y_true,y_pred,y_prob,y_score_pred,score
0,LFGRDLSY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,9.999058e-01,0,0.819
1,TDKKTHLY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,9.999999e-01,1,0.999
2,RSDTPLIY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000e+00,1,0.999
3,NSDLVQKY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000e+00,1,0.999
4,LSDLLDWK,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,9.996661e-01,1,0.999
...,...,...,...,...,...,...,...,...,...,...
169518,RVLLYNHNLL,10,HLA-B*44:27,0,YYTKYREISTNTYENTAYIRYDDYTWAVDAYLSY,0,0,1.109294e-11,0,0.003
169519,LGTIIFAHCQ,10,HLA-B*44:27,0,YYTKYREISTNTYENTAYIRYDDYTWAVDAYLSY,0,0,1.026913e-09,0,0.003
169520,PPASPTIIPR,10,HLA-B*44:27,0,YYTKYREISTNTYENTAYIRYDDYTWAVDAYLSY,0,0,1.143919e-05,0,0.003
169521,QPNHTPAGPP,10,HLA-B*44:27,0,YYTKYREISTNTYENTAYIRYDDYTWAVDAYLSY,0,0,4.892460e-09,0,0.003


In [22]:
merged_data_dict['ann']

,peptide,length,HLA,label,HLA_sequence,y_true,y_pred,y_prob,ic50,percentile_rank
0,LFGRDLSY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,9.999058e-01,15110.80,6.60
1,TDKKTHLY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,9.999999e-01,69.67,0.13
2,RSDTPLIY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000e+00,138.12,0.21
3,NSDLVQKY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000e+00,253.80,0.33
4,LSDLLDWK,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,9.996661e-01,12112.16,5.00
...,...,...,...,...,...,...,...,...,...,...
141681,LLPAGTYKV,9,HLA-B*83:01,0,YYSEYRNIYAQTDESNLYIRYDDYTWAVDAYLSY,0,0,1.028038e-09,19717.16,5.40
141682,LVPDVSYGP,9,HLA-B*83:01,0,YYSEYRNIYAQTDESNLYIRYDDYTWAVDAYLSY,0,0,3.147198e-12,21392.22,6.40
141683,TTVLPPIPA,9,HLA-B*83:01,0,YYSEYRNIYAQTDESNLYIRYDDYTWAVDAYLSY,0,0,8.193968e-03,19370.38,5.20
141684,VMLETLTLI,9,HLA-B*83:01,0,YYSEYRNIYAQTDESNLYIRYDDYTWAVDAYLSY,0,1,8.515308e-01,20224.27,5.70


# 总的平均结果

In [37]:
methods_performances_dict = {}
pHLAIformer_according_methods_performances_dict = {}
for method in (methods[:-1]):
    print('*** Type = {} | Method = {} ***'.format(ori_type, method))
    merged_data = merged_data_dict[method]
    pHLAIformer_according_methods_performances_dict[method] = performances(merged_data.y_true, merged_data.y_pred, merged_data.y_prob, rank_or_ic50 = False)
    merged_data, method_performance = benchmark_label_results(merged_data, method)
    methods_performances_dict[method] = method_performance

metrics = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']
methods_performances_pd = pd.DataFrame(methods_performances_dict, index = metrics).T
pHLAIformer_according_methods_performances_pd = pd.DataFrame(pHLAIformer_according_methods_performances_dict, index = metrics).T

methods_performances_pd.loc['pHLAIformer'] =  pHLAIformer_metrics_pd.loc[0]

*** Type = evaluation | Method = ann ***
tn = 65539, fp = 5156, fn = 5372, tp = 65619
y_pred: 0 = 70911 | 1 = 70775
y_true: 0 = 70695 | 1 = 70991
auc=0.9764|sensitivity=0.9243|specificity=0.9271|acc=0.9257|mcc=0.8514
precision=0.9271|recall=0.9243|f1=0.9257|aupr=0.9760
-----Performances according to IC50 threshold-----
tn = 69565, fp = 1130, fn = 30069, tp = 40922
y_pred: 0 = 99634 | 1 = 42052
y_true: 0 = 70695 | 1 = 70991
auc=0.9320|sensitivity=0.5764|specificity=0.9840|acc=0.7798|mcc=0.6134
precision=0.9731|recall=0.5764|f1=0.7240|aupr=0.9148
*** Type = evaluation | Method = comblib_sidney2008 ***
tn = 15229, fp = 1205, fn = 1413, tp = 15165
y_pred: 0 = 16642 | 1 = 16370
y_true: 0 = 16434 | 1 = 16578
auc=0.9732|sensitivity=0.9148|specificity=0.9267|acc=0.9207|mcc=0.8415
precision=0.9264|recall=0.9148|f1=0.9205|aupr=0.9732
-----Performances according to IC50 threshold-----
tn = 0, fp = 16434, fn = 0, tp = 16578
y_pred: 0 = 0 | 1 = 33012
y_true: 0 = 16434 | 1 = 16578
auc=0.8641|sensiti

In [83]:
pHLAIformer_according_methods_performances_pd.T

,ann,comblib_sidney2008,consensus,netmhccons,netmhcpan_ba,netmhcpan_el,netmhcstabpan,pickpocket,smm,smmpmbec,anthem
roc_auc,0.976444,0.973216,0.976307,0.978766,0.978880,0.978880,0.978880,0.978880,0.978769,0.978820,0.978904
accuracy,0.925695,0.920696,0.925285,0.930498,0.930844,0.930844,0.930844,0.930844,0.931560,0.931675,0.930959
mcc,0.851394,0.841464,0.850577,0.860996,0.861688,0.861688,0.861688,0.861688,0.863132,0.863361,0.861919
f1,0.925737,0.920541,0.925313,0.930586,0.930927,0.930927,0.930927,0.930927,0.931552,0.931672,0.931050
sensitivity,0.924328,0.914767,0.923690,0.929812,0.930341,0.930341,0.930341,0.930341,0.929181,0.929438,0.930546
specificity,0.927067,0.926676,0.926887,0.931186,0.931348,0.931348,0.931348,0.931348,0.933951,0.933923,0.931374
precision,0.927149,0.926390,0.926941,0.931362,0.931513,0.931513,0.931513,0.931513,0.933935,0.933917,0.931555
recall,0.924328,0.914767,0.923690,0.929812,0.930341,0.930341,0.930341,0.930341,0.929181,0.929438,0.930546
aupr,0.975990,0.973207,0.975867,0.978055,0.978183,0.978183,0.978183,0.978183,0.978110,0.978148,0.978227


In [82]:
methods_performances_pd.T

,ann,comblib_sidney2008,consensus,netmhccons,netmhcpan_ba,netmhcpan_el,netmhcstabpan,pickpocket,smm,smmpmbec,anthem,pHLAIformer
roc_auc,0.932007,0.864111,0.931086,0.952890,0.955066,0.955796,0.915652,0.924055,0.912403,0.915624,0.977069,0.978880
accuracy,0.779802,0.502181,0.788322,0.789359,0.801666,0.795115,0.789685,0.702044,0.788691,0.788739,0.923851,0.930844
mcc,0.613394,NaN,0.626864,0.629417,0.649185,0.643276,0.621612,0.492774,0.606827,0.609004,0.847720,0.861688
f1,0.724008,0.668603,0.737543,0.738655,0.757434,0.744978,0.743866,0.584021,0.751184,0.749755,0.924256,0.930927
sensitivity,0.576439,1.000000,0.593587,0.594095,0.618182,0.597420,0.609681,0.417556,0.636402,0.631468,0.927472,0.930341
specificity,0.984016,0.000000,0.983885,0.985448,0.985823,0.993537,0.970349,0.987576,0.941723,0.946752,0.920217,0.931348
precision,0.973129,0.502181,0.973679,0.976190,0.977661,0.989336,0.953784,0.971209,0.916482,0.922570,0.921062,0.931513
recall,0.576439,1.000000,0.593587,0.594095,0.618182,0.597420,0.609681,0.417556,0.636402,0.631468,0.927472,0.930341
aupr,0.914784,0.849830,0.909860,0.945905,0.944328,0.964232,0.923492,0.913137,0.912661,0.914233,0.977853,0.978183


In [42]:
pHLAIformer_according_methods_performances_pd - methods_performances_pd

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
ann,0.044436,0.145893,0.238000,0.201729,0.347889,-0.056949,-0.045979,0.347889,0.061206
anthem,0.001835,0.007108,0.014199,0.006794,0.003074,0.011158,0.010493,0.003074,0.000374
comblib_sidney2008,0.109105,0.418514,NaN,0.251939,-0.085233,0.926676,0.424209,-0.085233,0.123377
consensus,0.045221,0.136964,0.223713,0.187770,0.330103,-0.056998,-0.046738,0.330103,0.066007
netmhccons,0.025876,0.141139,0.231580,0.191931,0.335717,-0.054262,-0.044828,0.335717,0.032150
netmhcpan_ba,0.023814,0.129178,0.212504,0.173493,0.312159,-0.054475,-0.046148,0.312159,0.033855
netmhcpan_el,0.023085,0.135728,0.218413,0.185949,0.332922,-0.062189,-0.057823,0.332922,0.013951
netmhcstabpan,0.063228,0.141159,0.240076,0.187061,0.320660,-0.039001,-0.022271,0.320660,0.054692
pHLAIformer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pickpocket,0.054825,0.228800,0.368915,0.346906,0.512786,-0.056228,-0.039696,0.512786,0.065047


# 针对每个HLA的结果

In [56]:
pHLAIformer_methods_metrics_length_hla_dict, methods_metrics_length_hla_dict = {}, {}
for method in methods[:-1]:
    pHLAIformer_methods_metrics_length_hla_dict[method], methods_metrics_length_hla_dict[method] = {}, {}
    
    if method == 'smm' or method == 'smmpmbec':
        lengths = list(range(8, 12))
    elif method == 'comblib_sidney2008':
        lengths = [9]
    else:
        lengths = list(range(8, 15))
    
    if 'ic50' and 'y_pred_ic50' in merged_data_dict[method].columns: 
        m = ['ic50', 'y_pred_ic50']
    elif 'percentile_rank' and 'y_pred_rank' in merged_data_dict[method].columns: 
        m = ['percentile_rank', 'y_pred_rank']
    elif 'score' and 'y_pred_score' in merged_data_dict[method].columns: 
        m = ['score', 'y_pred_score']
     
    metrics_length_hla_dict, pHLAIformer_metrics_length_hla_dict = eval_metrics_length_hla_dict(merged_data_dict[method], lengths, m)
    pHLAIformer_methods_metrics_length_hla_dict[method][m[0]], methods_metrics_length_hla_dict[method][m[0]] = {}, {}
    for l in lengths:
        methods_metrics_length_hla_dict[method][m[0]][l]  = eval_performances_to_pd(metrics_length_hla_dict, l)
        pHLAIformer_methods_metrics_length_hla_dict[method][m[0]][l] = eval_performances_to_pd(pHLAIformer_metrics_length_hla_dict, l)

 30%|███       | 10/33 [00:00<00:00, 97.62it/s]

tn = 21, fp = 0, fn = 16, tp = 5
y_pred: 0 = 37 | 1 = 5
y_true: 0 = 21 | 1 = 21
auc=0.9410|sensitivity=0.2381|specificity=1.0000|acc=0.6190|mcc=0.3676
precision=1.0000|recall=0.2381|f1=0.3846|aupr=0.9509
tn = 20, fp = 1, fn = 0, tp = 21
y_pred: 0 = 20 | 1 = 22
y_true: 0 = 21 | 1 = 21
auc=0.9649|sensitivity=1.0000|specificity=0.9524|acc=0.9762|mcc=0.9535
precision=0.9545|recall=1.0000|f1=0.9767|aupr=0.9493
tn = 297, fp = 0, fn = 195, tp = 103
y_pred: 0 = 492 | 1 = 103
y_true: 0 = 297 | 1 = 298
auc=0.9794|sensitivity=0.3456|specificity=1.0000|acc=0.6723|mcc=0.4568
precision=1.0000|recall=0.3456|f1=0.5137|aupr=0.9758
tn = 290, fp = 7, fn = 11, tp = 287
y_pred: 0 = 301 | 1 = 294
y_true: 0 = 297 | 1 = 298
auc=0.9924|sensitivity=0.9631|specificity=0.9764|acc=0.9697|mcc=0.9396
precision=0.9762|recall=0.9631|f1=0.9696|aupr=0.9922
tn = 157, fp = 0, fn = 121, tp = 36
y_pred: 0 = 278 | 1 = 36
y_true: 0 = 157 | 1 = 157
auc=0.8920|sensitivity=0.2293|specificity=1.0000|acc=0.6146|mcc=0.3599
precisio

  0%|          | 0/75 [00:00<?, ?it/s]

tn = 34, fp = 0, fn = 23, tp = 11
y_pred: 0 = 57 | 1 = 11
y_true: 0 = 34 | 1 = 34
auc=0.9472|sensitivity=0.3235|specificity=1.0000|acc=0.6618|mcc=0.4393
precision=1.0000|recall=0.3235|f1=0.4889|aupr=0.9207
tn = 33, fp = 1, fn = 4, tp = 30
y_pred: 0 = 37 | 1 = 31
y_true: 0 = 34 | 1 = 34
auc=0.9784|sensitivity=0.8824|specificity=0.9706|acc=0.9265|mcc=0.8563
precision=0.9677|recall=0.8824|f1=0.9231|aupr=0.9829
tn = 45, fp = 0, fn = 45, tp = 0
y_pred: 0 = 90 | 1 = 0
y_true: 0 = 45 | 1 = 45
auc=0.9872|sensitivity=0.0000|specificity=1.0000|acc=0.5000|mcc=nan
precision=0.0000|recall=0.0000|f1=0.0000|aupr=0.9887
tn = 42, fp = 3, fn = 4, tp = 41
y_pred: 0 = 46 | 1 = 44
y_true: 0 = 45 | 1 = 45
auc=0.9881|sensitivity=0.9111|specificity=0.9333|acc=0.9222|mcc=0.8447
precision=0.9318|recall=0.9111|f1=0.9213|aupr=0.9878
tn = 228, fp = 3, fn = 64, tp = 167
y_pred: 0 = 292 | 1 = 170
y_true: 0 = 231 | 1 = 231
auc=0.9854|sensitivity=0.7229|specificity=0.9870|acc=0.8550|mcc=0.7361
precision=0.9824|recall=

 13%|█▎        | 10/75 [00:00<00:01, 45.69it/s]

...

tn = 58, fp = 4, fn = 4, tp = 58
y_pred: 0 = 62 | 1 = 62
y_true: 0 = 62 | 1 = 62
auc=0.9465|sensitivity=0.9355|specificity=0.9355|acc=0.9355|mcc=0.8710
precision=0.9355|recall=0.9355|f1=0.9355|aupr=0.9596
tn = 55, fp = 7, fn = 3, tp = 59
y_pred: 0 = 58 | 1 = 66
y_true: 0 = 62 | 1 = 62
auc=0.9770|sensitivity=0.9516|specificity=0.8871|acc=0.9194|mcc=0.8405
precision=0.8939|recall=0.9516|f1=0.9219|aupr=0.9790
tn = 43, fp = 0, fn = 4, tp = 39
y_pred: 0 = 47 | 1 = 39
y_true: 0 = 43 | 1 = 43
auc=0.9730|sensitivity=0.9070|specificity=1.0000|acc=0.9535|mcc=0.9109
precision=1.0000|recall=0.9070|f1=0.9512|aupr=0.9848
tn = 42, fp = 1, fn = 2, tp = 41
y_pred: 0 = 44 | 1 = 42
y_true: 0 = 43 | 1 = 43
auc=0.9957|sensitivity=0.9535|specificity=0.9767|acc=0.9651|mcc=0.9305
precision=0.9762|recall=0.9535|f1=0.9647|aupr=0.9960
tn = 30, fp = 6, fn = 6, tp = 30
y_pred: 0 = 36 | 1 = 36
y_true: 0 = 36 | 1 = 36
auc=0.9275|sensitivity=0.8333|specificity=0.8333|acc=0.8333|mcc=0.6667
precision=0.8333|recall=0.83

  0%|          | 0/13 [00:00<?, ?it/s]

tn = 11, fp = 3, fn = 2, tp = 12
y_pred: 0 = 13 | 1 = 15
y_true: 0 = 14 | 1 = 14
auc=0.9184|sensitivity=0.8571|specificity=0.7857|acc=0.8214|mcc=0.6445
precision=0.8000|recall=0.8571|f1=0.8276|aupr=0.9355
tn = 25, fp = 2, fn = 3, tp = 24
y_pred: 0 = 28 | 1 = 26
y_true: 0 = 27 | 1 = 27
auc=0.9890|sensitivity=0.8889|specificity=0.9259|acc=0.9074|mcc=0.8154
precision=0.9231|recall=0.8889|f1=0.9057|aupr=0.9890
tn = 27, fp = 0, fn = 1, tp = 26
y_pred: 0 = 28 | 1 = 26
y_true: 0 = 27 | 1 = 27
auc=1.0000|sensitivity=0.9630|specificity=1.0000|acc=0.9815|mcc=0.9636
precision=1.0000|recall=0.9630|f1=0.9811|aupr=1.0000
tn = 118, fp = 44, fn = 32, tp = 130
y_pred: 0 = 150 | 1 = 174
y_true: 0 = 162 | 1 = 162
auc=0.8901|sensitivity=0.8025|specificity=0.7284|acc=0.7654|mcc=0.5323
precision=0.7471|recall=0.8025|f1=0.7738|aupr=0.9138
tn = 140, fp = 22, fn = 28, tp = 134
y_pred: 0 = 168 | 1 = 156
y_true: 0 = 162 | 1 = 162
auc=0.9200|sensitivity=0.8272|specificity=0.8642|acc=0.8457|mcc=0.6918
precision=0.

100%|██████████| 13/13 [00:00<00:00, 114.81it/s]


tn = 27, fp = 5, fn = 4, tp = 28
y_pred: 0 = 31 | 1 = 33
y_true: 0 = 32 | 1 = 32
auc=0.9404|sensitivity=0.8750|specificity=0.8438|acc=0.8594|mcc=0.7191
precision=0.8485|recall=0.8750|f1=0.8615|aupr=0.9560
tn = 13, fp = 2, fn = 7, tp = 8
y_pred: 0 = 20 | 1 = 10
y_true: 0 = 15 | 1 = 15
auc=0.8644|sensitivity=0.5333|specificity=0.8667|acc=0.7000|mcc=0.4243
precision=0.8000|recall=0.5333|f1=0.6400|aupr=0.8700
tn = 12, fp = 3, fn = 3, tp = 12
y_pred: 0 = 15 | 1 = 15
y_true: 0 = 15 | 1 = 15
auc=0.8889|sensitivity=0.8000|specificity=0.8000|acc=0.8000|mcc=0.6000
precision=0.8000|recall=0.8000|f1=0.8000|aupr=0.8515
tn = 13, fp = 3, fn = 1, tp = 15
y_pred: 0 = 14 | 1 = 18
y_true: 0 = 16 | 1 = 16
auc=0.9316|sensitivity=0.9375|specificity=0.8125|acc=0.8750|mcc=0.7559
precision=0.8333|recall=0.9375|f1=0.8824|aupr=0.9450
tn = 13, fp = 3, fn = 1, tp = 15
y_pred: 0 = 14 | 1 = 18
y_true: 0 = 16 | 1 = 16
auc=0.9688|sensitivity=0.9375|specificity=0.8125|acc=0.8750|mcc=0.7559
precision=0.8333|recall=0.937

In [88]:
length_methods_dict = {}
for length in range(8,15):
    if length == 9: 
        length_methods_dict[length] = ['transformer', 'ann', 'consensus', 'netmhccons', 'netmhcpan_ba', 'netmhcpan_el', 'netmhcstabpan', 'pickpocket', 'smm', 'smmpmbec', 'anthem', 'comblib_sidney2008']
    elif length in range(12,15): 
        length_methods_dict[length] = ['transformer', 'ann', 'consensus', 'netmhccons', 'netmhcpan_ba', 'netmhcpan_el', 'netmhcstabpan', 'pickpocket', 'anthem']
    else: 
        length_methods_dict[length] = ['transformer',  'ann', 'consensus', 'netmhccons', 'netmhcpan_ba', 'netmhcpan_el', 'netmhcstabpan', 'pickpocket', 'smm', 'smmpmbec', 'anthem']

In [89]:
length_methods_newname_dict = {}
for length in range(8,15):
    if length == 9: 
        length_methods_newname_dict[length] = ['pHLAIformer', 'ANN', 'Consensus', 'NetMHCcons', 'NetMHCpan_BA', 'NetMHCpan_EL', 'NetMHCstabpan', 'PickPocket', 'SMM', 'SMMPMBEC', 'Anthem', 'CombLib']
    elif length in range(12,15): 
        length_methods_newname_dict[length] = ['pHLAIformer', 'ANN', 'Consensus', 'NetMHCcons', 'NetMHCpan_BA', 'NetMHCpan_EL', 'NetMHCstabpan', 'PickPocket', 'Anthem']
    else: 
        length_methods_newname_dict[length] = ['pHLAIformer', 'ANN', 'Consensus', 'NetMHCcons', 'NetMHCpan_BA', 'NetMHCpan_EL', 'NetMHCstabpan', 'PickPocket', 'SMM', 'SMMPMBEC', 'Anthem']

In [164]:
cols = ['roc_auc', 'accuracy', 'mcc', 'f1']
length, col = 14, cols[3]

length_methods = length_methods_dict[length]
length_methods_newname = length_methods_newname_dict[length]
res_pd = pHLAIformer_metrics_length_hla_pd_dict[length][col]
for method in length_methods[1:]:
    if 'ic50' in methods_metrics_length_hla_dict[method].keys(): 
        m = 'ic50'
    elif 'percentile_rank' in methods_metrics_length_hla_dict[method].keys(): 
        m = 'percentile_rank'
    elif 'score' in methods_metrics_length_hla_dict[method].keys(): 
        m = 'score'
        
    if method != 'anthem':
        temp = methods_metrics_length_hla_dict[method][m][length][col]
    else:
        temp = methods_metrics_length_hla_dict[method][m][length][col]
    res_pd = pd.concat([res_pd, temp], axis = 1)
res_pd.columns = length_methods_newname
print([res_pd[col].count()-3 for col in res_pd.columns])
print(res_pd.iloc[:-4, :].min().min(), res_pd.iloc[:-4, :].max().max())
res_pd

[13, 12, 12, 13, 13, 13, 13, 13, 13]
0.0 0.9696969696969697


,pHLAIformer,ANN,Consensus,NetMHCcons,NetMHCpan_BA,NetMHCpan_EL,NetMHCstabpan,PickPocket,Anthem
HLA-A*68:02,0.769231,0.105263,0.105263,0.363636,0.105263,0.105263,0.434783,0.105263,0.666667
HLA-B*15:01,0.861538,0.638298,0.666667,0.877193,0.608696,0.666667,0.877193,0.222222,0.906250
HLA-B*27:09,0.833333,NaN,NaN,0.437500,0.076923,0.275862,0.275862,0.148148,0.716981
HLA-B*07:02,0.898551,0.909091,0.941176,0.892308,0.758621,0.870968,0.909091,0.838710,0.911765
HLA-B*57:01,0.751445,0.516129,0.571429,0.528926,0.452174,0.516667,0.524590,0.424779,0.725275
HLA-B*35:01,0.800000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.640000
HLA-C*04:01,0.772727,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,0.000000,0.645161
HLA-C*06:02,0.666667,0.000000,0.100000,0.000000,0.000000,0.000000,0.054054,0.000000,0.707692
HLA-A*24:02,0.882353,0.476190,0.583333,0.583333,0.222222,0.666667,0.740741,0.400000,0.882353
HLA-A*01:01,0.941176,0.853933,0.941176,0.879121,0.756098,0.969697,0.905263,0.111111,0.969697
